In [1]:
import requests
import pandas as pd
import json
import Keys
import time
import pickle
import re
from rauth import OAuth1Service

In [2]:
def get_musix_genres(key):
    url = 'https://api.musixmatch.com/ws/1.1/'
    sub_url = 'music.genres.get'
    params = {
        'apikey': key
    }
    
    musix_genres_json = requests.get(url + sub_url, params = params).json()['message']['body']['music_genre_list']
    
    musix_genres = []
    
    for dictionary in musix_genres_json:
        genre = dictionary['music_genre']
        musix_genres.append({'genre_id': genre['music_genre_id'],
                             'genre_name': genre['music_genre_vanity']})
    
    return musix_genres

In [3]:
def get_musix_track_info_by_genre(genres, key, id_limit = 5000):
    url = 'https://api.musixmatch.com/ws/1.1/'
    sub_url = 'track.search'
    
    for genre in genres:
        params = {
            'apikey': key,
            'q_track': '*',
            'f_music_genre_id': genre['genre_id'],
            'f_has_lyrics': 1,
            'f_lyrics_language': 'en',
            'page_size': 100,
            'page': 1
        }
        
        num_tracks = requests.get(url + sub_url, params = params).json()['message']['header']['available']
        page_limit = min(num_tracks, id_limit)
        page_max = (page_limit // 100) + 1
        pages = range(2, page_max)
        
        print('{}: Retrieving {} ids in {} pages'.format(genre['genre_name'], page_limit, page_max - 1))
        print('Retrieving page 1 of {}'.format(page_max - 1))
        
        tracks = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']
        
        with open('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', 'a') as f:
            track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                    track['track']['track_name'],
                                                                    genre['genre_id'],
                                                                    genre['genre_name'],
                                                                    track['track']['album_id'],
                                                                    track['track']['album_name'],
                                                                    track['track']['artist_id'],
                                                                    track['track']['artist_name'])
                          for track in tracks]
                         
            f.writelines(track_info)

        for page in pages:
            time.sleep(2.5)
            params['page'] = page
            print('Retrieving page {} of {}'.format(page, page_max - 1))
            tracks = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']
            
            with open('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', 'a') as f:
                track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                        track['track']['track_name'],
                                                                        genre['genre_id'],
                                                                        genre['genre_name'],
                                                                        track['track']['album_id'],
                                                                        track['track']['album_name'],
                                                                        track['track']['artist_id'],
                                                                        track['track']['artist_name'])
                              for track in tracks]

                f.writelines(track_info)
    
        print('Retrieved {} ids for genre {}'.format(page_limit, genre['genre_name']))
    
    return

In [4]:
musixmatch_url = 'https://api.musixmatch.com/ws/1.1/'
musixmatch_key = Keys.musixmatch_key

In [71]:
with open('../data/raw/genres_dict.pickle', 'rb') as f:
    musix_genres = pickle.load(f)

In [74]:
musix_genres

[{'genre_id': 34, 'genre_name': 'Music'},
 {'genre_id': 1210, 'genre_name': 'Blues-Acoustic-Blues'},
 {'genre_id': 1007, 'genre_name': 'Blues-Chicago-Blues'},
 {'genre_id': 1009, 'genre_name': 'Blues-Classic-Blues'},
 {'genre_id': 1010, 'genre_name': 'Blues-Contemporary-Blues'},
 {'genre_id': 1011, 'genre_name': 'Blues-Country-Blues'},
 {'genre_id': 1012, 'genre_name': 'Blues-Delta-Blues'},
 {'genre_id': 1013, 'genre_name': 'Blues-Electric-Blues'},
 {'genre_id': 1167, 'genre_name': 'Comedy-Novelty'},
 {'genre_id': 1171, 'genre_name': 'Comedy-Standup-Comedy'},
 {'genre_id': 1014, 'genre_name': "Children's-Music-Lullabies"},
 {'genre_id': 1015, 'genre_name': "Children's-Music-Sing-Along"},
 {'genre_id': 1016, 'genre_name': "Children's-Music-Stories"},
 {'genre_id': 1024, 'genre_name': 'Classical-Impressionist'},
 {'genre_id': 1025, 'genre_name': 'Classical-Medieval-Era'},
 {'genre_id': 1026, 'genre_name': 'Classical-Minimalism'},
 {'genre_id': 1027, 'genre_name': 'Classical-Modern-Era'},

In [126]:
genre_ids_to_keep = [2, 6, 11, 14, 15, 17, 18, 20, 22, 24, 1149, 1152]
# top_level_genres = sorted([x for x in musix_genres if x['genre_id'] < 54], key = lambda x: x['genre_id'])
# genre_ids_to_drop = [0, 3, 4, 9, 10, 16, 25, 27, 28, 29, 30, 34, 50, 51, 52, 53]
# genres_to_extract = [x for x in top_level_genres if x['genre_id'] not in genre_ids_to_drop]
genres_to_extract = [x for x in musix_genres if x['genre_id'] in genre_ids_to_keep]

In [129]:
genres_to_extract

[{'genre_id': 1149, 'genre_name': 'Rock-Death-Metal-Black-Metal'},
 {'genre_id': 1152, 'genre_name': 'Rock-Hard-Rock'},
 {'genre_id': 2, 'genre_name': 'Blues'},
 {'genre_id': 6, 'genre_name': 'Country'},
 {'genre_id': 11, 'genre_name': 'Jazz'},
 {'genre_id': 14, 'genre_name': 'Pop'},
 {'genre_id': 15, 'genre_name': 'R-B-Soul'},
 {'genre_id': 17, 'genre_name': 'Dance'},
 {'genre_id': 18, 'genre_name': 'Hip-Hop-Rap'},
 {'genre_id': 20, 'genre_name': 'Alternative'},
 {'genre_id': 22, 'genre_name': 'Christian-Gospel'},
 {'genre_id': 24, 'genre_name': 'Reggae'}]

In [138]:
# get_musix_track_info_by_genre([genres_to_extract[0]], musixmatch_key, id_limit = 10_000)
get_musix_track_info_by_genre(genres_to_extract[1:], musixmatch_key, id_limit = 10_000)

Rock-Hard-Rock: Retrieving 10000 ids in 100 pages
Retrieving page 1 of 100
Retrieving page 2 of 100
Retrieving page 3 of 100
Retrieving page 4 of 100
Retrieving page 5 of 100
Retrieving page 6 of 100
Retrieving page 7 of 100
Retrieving page 8 of 100
Retrieving page 9 of 100
Retrieving page 10 of 100
Retrieving page 11 of 100
Retrieving page 12 of 100
Retrieving page 13 of 100
Retrieving page 14 of 100
Retrieving page 15 of 100
Retrieving page 16 of 100
Retrieving page 17 of 100
Retrieving page 18 of 100
Retrieving page 19 of 100
Retrieving page 20 of 100
Retrieving page 21 of 100
Retrieving page 22 of 100
Retrieving page 23 of 100
Retrieving page 24 of 100
Retrieving page 25 of 100
Retrieving page 26 of 100
Retrieving page 27 of 100
Retrieving page 28 of 100
Retrieving page 29 of 100
Retrieving page 30 of 100
Retrieving page 31 of 100
Retrieving page 32 of 100
Retrieving page 33 of 100
Retrieving page 34 of 100
Retrieving page 35 of 100
Retrieving page 36 of 100
Retrieving page 37 of 1

Retrieving page 6 of 100
Retrieving page 7 of 100
Retrieving page 8 of 100
Retrieving page 9 of 100
Retrieving page 10 of 100
Retrieving page 11 of 100
Retrieving page 12 of 100
Retrieving page 13 of 100
Retrieving page 14 of 100
Retrieving page 15 of 100
Retrieving page 16 of 100
Retrieving page 17 of 100
Retrieving page 18 of 100
Retrieving page 19 of 100
Retrieving page 20 of 100
Retrieving page 21 of 100
Retrieving page 22 of 100
Retrieving page 23 of 100
Retrieving page 24 of 100
Retrieving page 25 of 100
Retrieving page 26 of 100
Retrieving page 27 of 100
Retrieving page 28 of 100
Retrieving page 29 of 100
Retrieving page 30 of 100
Retrieving page 31 of 100
Retrieving page 32 of 100
Retrieving page 33 of 100
Retrieving page 34 of 100
Retrieving page 35 of 100
Retrieving page 36 of 100
Retrieving page 37 of 100
Retrieving page 38 of 100
Retrieving page 39 of 100
Retrieving page 40 of 100
Retrieving page 41 of 100
Retrieving page 42 of 100
Retrieving page 43 of 100
Retrieving page 

TypeError: list indices must be integers or slices, not str

In [139]:
url = 'https://api.musixmatch.com/ws/1.1/'
sub_url = 'track.search'
params = {
    'apikey': musixmatch_key,
    'q_track': '*',
#     'f_music_genre_id': None,
    'f_has_lyrics': 1,
    'f_lyrics_language': 'en',
    'page_size': 100,
    'page': 1
    }

# for genre in genres_to_extract:
#     params['f_music_genre_id'] = genre['genre_id']
#     print('{}: {} available'.format(genre['genre_name'],
#                         requests.get(url + sub_url, params = params).json()['message']['header']['available']))

test = requests.get(url + sub_url, params = params).json()
# test = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']

In [131]:
url = 'https://api.musixmatch.com/ws/1.1/'
sub_url = 'track.search'
params = {
    'apikey': musixmatch_key,
    'q_track': '*',
    'f_music_genre_id': 1134,
    'f_has_lyrics': 1,
    'f_lyrics_language': 'en',
    'page_size': 100,
    'page': 1
}

print('{}: {} available'.format('Pop-Teen-Pop',
                    requests.get(url + sub_url, params = params).json()['message']['header']['available']))

Pop-Teen-Pop: 2263 available


In [140]:
test

{'message': {'header': {'status_code': 401,
   'execute_time': 0.00035190582275391},
  'body': []}}

In [105]:
track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                    track['track']['track_name'],
                                                                    13,
                                                                    'New-Age',
                                                                    track['track']['album_id'],
                                                                    track['track']['album_name'],
                                                                    track['track']['artist_id'],
                                                                    track['track']['artist_name'])
                          for track in test]

In [106]:
track_info

['52786577\tIsland Rhythms - Relaxing Tropical Shoreline\t13\tNew-Age\t17034085\tWeekend Getaway (Nature Sound With Music)\t26396438\tNaturescape Music\n',
 '69646327\tWhat You Mean to Me\t13\tNew-Age\t19179579\t50 Musak Tracks\t27558642\tBackground Music Soundscape\n',
 '51629081\tBeautiful Night\t13\tNew-Age\t16913776\tVoyage, Vol. 2\t26325478\tBlue Synthax\n',
 '82793288\tMiracle Worker\t13\tNew-Age\t20758691\tAwesome Nature Sounds – Meditation Music for Deep Concentration, Pure Mind, Relaxation Techniques for Anxiety, Motivation, Calming Music for Mental Inspiration\t28663090\tAwesome Nature Sounds Ensemble\n',
 "80167262\tLes six couleurs de l'amour\t13\tNew-Age\t20448298\tLove\t28466590\tLeo Bormans\n",
 '68256576\tKarma\t13\tNew-Age\t18989768\tKarma\t27025600\tClaudio Barria Casanueva\n',
 '98206876\t달님과 별님의 속삭임 The Whisper of Moon And Star\t13\tNew-Age\t21884509\t숲속의 아기 자장가 Lullaby In the Forest\t30701495\tPink Music\n',
 "48101364\tDreamin' In the Water\t13\tNew-Age\t16543677\

In [ ]:
# genre = {'genre_id': 13, 'genre_name': 'New-Age'}

In [ ]:
# with open('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', 'a') as f:
#     track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
#                                                                             track['track']['track_name'],
#                                                                             genre['genre_id'],
#                                                                             genre['genre_name'],
#                                                                             track['track']['album_id'],
#                                                                             track['track']['album_name'],
#                                                                             track['track']['artist_id'],
#                                                                             track['track']['artist_name'])
#                   for track in test]
#     f.writelines(track_info)

In [ ]:
# track_info

In [ ]:
# musix_genres = get_musix_genres(musixmatch_key)

# with open('../data/raw/genres_dict.pickle', 'wb') as f:
#     pickle.dump(musix_genres, f)

In [ ]:
with open('../data/raw/musix_match_track_data.csv', 'r') as f:
    csv_data = f.read()

In [ ]:
csv_data_split = csv_data.split('\n')
    

In [ ]:
for index, line in enumerate(csv_data_split):
    line_split = line.split(',')
#     csv_data_split[index] = re.sub(r'([A-Za-z][\w, ]+),', '\"' + "\g<1>" + '\",', line)
    
    if len(line_split) > 4:
        movie_title = "\"{}\"".format(' '.join(line_split[1:len(line_split) - 2]))
        csv_data_split[index] = ','.join([line_split[0], movie_title, line_split[-2], line_split[-1]])

In [ ]:
csv_data_split[24]

In [ ]:
with open('../data/raw/musix_match_track_data_cleaned.csv', 'w') as f:
    for line in csv_data_split:
        f.write(line + '\n')

In [9]:
df = pd.read_csv('../data/raw/musix_match_track_data_cleaned.csv', header = None)
df.columns = ['title_id', 'title_name', 'genre_id', 'genre_name']

In [76]:
df.genre_name.value_counts()

Blues               10000
New-Age             10000
Rock                10000
Reggae              10000
Electronic           9999
Dance                9999
Hip-Hop-Rap          9999
World                9999
Christian-Gospel     9999
R-B-Soul             9999
Holiday              9998
Jazz                 9997
Vocal                9996
Country              9995
Alternative          9993
Pop                  9989
Latin                8000
Classical            6899
Name: genre_name, dtype: int64

In [130]:
df.loc[df.genre_name == 'Pop']

,title_id,title_name,genre_id,genre_name
74888,78395374,Ты не моё,14,Pop
74889,59701326,Back To December (Vocal Version),14,Pop
74890,62708557,Writing On the Wall,14,Pop
74891,79815007,Do You Know the Way to San José,14,Pop
74892,76570084,Pink and Blue,14,Pop
...,...,...,...,...
84872,128775234,Radio (Radio Edit),14,Pop
84873,107413016,Last Kiss 2006,14,Pop
84874,100457346,Paranoia,14,Pop
84875,72910894,Plane Ride,14,Pop


In [141]:
df2 = pd.read_csv('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', sep = '\t', header = None)
df2.columns = ['title_id', 'title_name', 'genre_id', 'genre_name',
               'album_id', 'album_name', 'artist_id', 'artist_name']

In [182]:
df2.genre_name.value_counts()

Rock-Death-Metal-Black-Metal    10000
Rock-Hard-Rock                  10000
Blues                           10000
Jazz                             9999
Country                          9997
Pop                              1400
Name: genre_name, dtype: int64

In [185]:
def scrape_song_lyrics(df, key, file):
    genius = Genius(key)
    
    for row in df.index:
        title = df.iloc[row].title_name
        artist = df.iloc[row].artist_name
        try:
            song = genius.search_song(title, artist)
        except Exception:
            continue
            
        if song is None:
            continue
        
        song_split = song.lyrics.split('\n')
        song_split_no_empty_headers = [x for x in song_split if '(' not in x and x != '' and '[' not in x]
        song_scraped_artefacts_removed = []

        for line in song_split_no_empty_headers:
            song_scraped_artefacts_removed.append(re.sub(r'([A-Z]?[a-z]+)[A-Z][\w\W]+', '\g<1>', line))

        lyrics = '|'.join(song_scraped_artefacts_removed)

        with open(file, 'a') as f:
            f.write('{}\t{}\t{}\n'.format(lyrics, title, artist))
            
    return

In [172]:
rock_death_black = df2.loc[df2.genre_name == 'Rock-Death-Metal-Black-Metal']

In [170]:
rock_death_black

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,8401936,Onward to Die,1149,Rock-Death-Metal-Black-Metal,10948290,Scandinavian Warmachine,168822,Paganizer
1,119125944,Illumina,1149,Rock-Death-Metal-Black-Metal,24582909,Humagination,217894,Exlibris
2,3602021,History Bodies,1149,Rock-Death-Metal-Black-Metal,10514844,Abominable,136857,Amagortis
3,6394110,An Eternal Dark Horizon,1149,Rock-Death-Metal-Black-Metal,10761256,An Eternal Dark Horizon,234614,Throne Of Katarsis
4,8401931,An Icon for the Damned,1149,Rock-Death-Metal-Black-Metal,10948290,Scandinavian Warmachine,168822,Paganizer
...,...,...,...,...,...,...,...,...
9995,70232335,Radiation Damage,1149,Rock-Death-Metal-Black-Metal,19270939,Wrought From Anguish,25446897,Human Debris
9996,79417522,Devourer of Worlds,1149,Rock-Death-Metal-Black-Metal,20359725,Rage of the Bloodbeast,89182,Debauchery
9997,163995659,Atheist,1149,Rock-Death-Metal-Black-Metal,31176568,Atheist,28422604,Warshipper
9998,156708372,Immensity,1149,Rock-Death-Metal-Black-Metal,29869359,Silent Witness,36506410,Aeolian


In [198]:
rock_hard_rock = df2.loc[df2.genre_name == 'Rock-Hard-Rock'].reset_index().drop('index', axis = 1)

In [199]:
rock_hard_rock

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,6594096,Redemptive Soul,1152,Rock-Hard-Rock,10779044,Stranger,326145,Ambitions
1,75833960,Death Rock,1152,Rock-Hard-Rock,19873499,Let's Get Serious,469513,Outloud
2,107807649,Run Baby Run,1152,Rock-Hard-Rock,22998451,Dead Ghost Run,31570758,External Use
3,83212274,Bridge of Regret,1152,Rock-Hard-Rock,20783863,Bridge of Regret,28687393,Protheus
4,70940495,Blacklisted,1152,Rock-Hard-Rock,19365867,Motheredland,27658099,Blakc
...,...,...,...,...,...,...,...,...
9995,138328788,Sororicide,1152,Rock-Hard-Rock,26996595,The Entity,27807316,Sororicide
9996,104958817,Bite the Dust,1152,Rock-Hard-Rock,22603355,The Messenger,68102,Casey Jones
9997,45967539,On Symmetry Pebbles,1152,Rock-Hard-Rock,16322526,"Dance a While, Upset",46412,Settlefish
9998,164242615,Bite the Bullet (United),1152,Rock-Hard-Rock,31228464,The Ultimate Collection Box Set,13293,Harem Scarem


In [200]:
blues = df2.loc[df2.genre_name == 'Blues'].reset_index().drop('index', axis = 1)

In [201]:
blues

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,47370391,Telephone Wires,2,Blues,16470466,Two To The Chest To Remember One To The Head T...,53617,Feeling Left Out
1,76806842,New York City Dry,2,Blues,20033902,Baby Please Don't Go,6086,Johnny Copeland
2,75175204,Passing Fancy,2,Blues,19808445,Passing Fancy,39802,Vaughn Monroe
3,50184104,I've Got Ringht to Love My Baby,2,Blues,16760431,King of the Blues,8876,B.B. King
4,57191935,Down The Road A Piece,2,Blues,17490237,Playboy Boogie,213471,Big Town Playboys
...,...,...,...,...,...,...,...,...
9995,876058,Up the Line,2,Blues,10292998,Blue Moon,13685,Robben Ford
9996,47590589,Long Time,2,Blues,16491822,Country Blues,1635,Lightnin' Hopkins
9997,70357462,Lonesome (Remastered),2,Blues,19290973,All the Best Songs (Fantastic Masterpieces Rem...,15841,Big Bill Broonzy
9998,114658175,Look At My Baby,2,Blues,23958084,The Chess Box,4292,Bo Diddley


In [181]:
scrape_song_lyrics(rock_death_black, genius_access_token)

Searching for "Onward to Die" by Paganizer...
Done.
Searching for "Illumina" by Exlibris...
Done.
Searching for "History Bodies" by Amagortis...
Searching for "An Eternal Dark Horizon" by Throne Of Katarsis...
Done.
Searching for "An Icon for the Damned" by Paganizer...
Done.
Searching for "Slaughtered Corpse" by Amagortis...
Done.
Searching for "Lost Children in the World" by Opus Doria...
Done.
Searching for "Gorroneality" by Avulsed...
No results found for: 'Gorroneality Avulsed'
Searching for "Heraldic" by Autumnblaze...
Done.
Searching for "Iced" by Bodyfarm...
Done.
Searching for "Remembrances" by Blessed in Sin...
Done.
Searching for "Soultheft" by Centurian...
Done.
Searching for "Bonesaw Bonanza" by Paganizer...
Done.
Searching for "Departure" by Eye of Solitude...
Done.
Searching for "Departure of the Stellar Fleet Marks the Year One" by Deconstructing Sequence...
No results found for: 'Departure of the Stellar Fleet Marks the Year One Deconstructing Sequence'
Searching for "

Done.
Searching for "Phantoms" by Nominon...
Done.
Searching for "Below All That Is Mortal" by Fragments of Unbecoming...
Done.
Searching for "Fatum (III Fate - Noodlot)" by Ancient Rites...
No results found for: 'Fatum (III Fate - Noodlot) Ancient Rites'
Searching for "He Sells Agony" by Abaddon Incarnate...
Done.
Searching for "Finger on the Trigger" by Voices of Decay...
Done.
Searching for "When the Solstice Reaches the Apogee" by Mythological Cold Towers...
No results found for: 'When the Solstice Reaches the Apogee Mythological Cold Towers'
Searching for "The Angels" by Dying Suffocation...
Done.
Searching for "Requiem on Their Graveth" by Arkha Sva...
Done.
Searching for "Beyond Their Lies" by Wedingoth...
No results found for: 'Beyond Their Lies Wedingoth'
Searching for "Into Red" by For Ruin...
Done.
Searching for "Ethnic Cleansing" by Inner Thought...
Done.
Searching for "Morbid Panzer Batallion" by Paganizer...
No results found for: 'Morbid Panzer Batallion Paganizer'
Search

Done.
Searching for "Death Comes in March" by ToDieFor...
Done.
Searching for "End of Tears" by ToDieFor...
Done.
Searching for "Framed Living Bust" by Castrensis...
Done.
Searching for "Unholy Soul" by Stefano Zani's Project...
No results found for: 'Unholy Soul Stefano Zani's Project'
Searching for "Interlude : Become of Me" by Enchantya...
Done.
Searching for "Disciples of the Hex" by Damnations Hammer...
Done.
Searching for "Hibernation Sun" by Fromhell...
No results found for: 'Hibernation Sun Fromhell'
Searching for "Shedding Liquid Flesh" by Fetocide...
No results found for: 'Shedding Liquid Flesh Fetocide'
Searching for "Cry for Love" by ToDieFor...
Done.
Searching for "Unknown III" by ToDieFor...
No results found for: 'Unknown III ToDieFor'
Searching for "Oblivion: Vision" by ToDieFor...
Done.
Searching for "Letters from Death Row, Pt. 1" by Stefano Zani's Project...
No results found for: 'Letters from Death Row, Pt. 1 Stefano Zani's Project'
Searching for "Insanity's Edge" by

Done.
Searching for "Yet I Breathe" by Eye of Solitude...
Done.
Searching for "Gathering the Stones" by Fen...
Done.
Searching for "March of Demise" by Kormak...
No results found for: 'March of Demise Kormak'
Searching for "Epitaph" by Messiah...
Done.
Searching for "Not A Nightmare" by Artillery...
Done.
Searching for "Lurid Assimilation" by Defeated Sanity...
Done.
Searching for "Divide et Impera" by Lahmia...
Done.
Searching for "Unplug My Brain" by Asylum Pyre...
Done.
Searching for "My Last Call" by Slechtvalk...
Done.
Searching for "It Comes in Whispers" by Leviathan...
Done.
Searching for "Caixa Pensatória" by Dallian...
No results found for: 'Caixa Pensatória Dallian'
Searching for "Broken Mirror" by Ashes of Moon...
Done.
Searching for "Under a Moonlit Sky" by Slechtvalk...
Done.
Searching for "Zealous Procreation" by Hetroertzen...
Done.
Searching for "Wounds of Christ" by Liturgy...
Done.
Searching for "Violence Reigns Supreme" by Winter Of Sin...
Done.
Searching for "Mephis

Done.
Searching for "Horrified by Repulsion" by Avulsed...
Done.
Searching for "Perfidia" by The Kennedy Veil...
Done.
Searching for "Towards The Atmospheric" by Crows As Divine...
Done.
Searching for "Rite of Passage" by Ancestors Blood...
Done.
Searching for "Born but Buried, I Can See the Light" by Intestine Baalism...
Done.
Searching for "Brethren of the Cross" by The Infernal Sea...
Done.
Searching for "Fetal Scabs" by Lividity...
Done.
Searching for "Jackpot" by Pokerface...
Done.
Searching for "The Wolves Have Fed Tonight" by Claret Ash...
No results found for: 'The Wolves Have Fed Tonight Claret Ash'
Searching for "Unchained" by Burden of Grief...
Done.
Searching for "Within the Realm of a Dying Sun (Live)" by Love Like Blood...
Done.
Searching for "Black Swan Events" by I Shalt Become...
Done.
Searching for "Underwatermoon" by Winterhorde...
Done.
Searching for "Take My Hand" by Heavenwood...
Done.
Searching for "Thy Baptism" by Arkha Sva...
Done.
Searching for "Monolith" by E

...

Done.
Searching for "The Key is Inside the Wall" by Darkthrone...
Done.
Searching for "Death Whispered a Lullaby" by Opeth...
Done.
Searching for "War Born (Tri-adverserenade)" by Daath...
Done.
Searching for "The Demons in Me" by Savn...
Done.
Searching for "Revolt" by Gorefest...
Done.
Searching for "Ravishing Grimness" by Darkthrone...
Done.
Searching for "Save Our Souls" by Clawfinger...
Done.
Searching for "Too Late to Call an Ambulance" by Psychonaut 4...
Done.
Searching for "Useless Sacrifice" by Death Decline...
Done.
Searching for "Shit Comes Brown" by Krabathor...
Done.
Searching for "Spawn of Flesh" by God Macabre...
Done.
Searching for "Souls of the Evil Departed" by At the Gates...
Done.
Searching for "Satan Redeemer" by Eosphorus...
No results found for: 'Satan Redeemer Eosphorus'
Searching for "The Link" by Gojira...
Done.
Searching for "No Amen" by Iskald...
Done.
Searching for "Nightmares of an Ether Drinker" by Cradle of Filth...
Done.
Searching for "Moment of Fright"

No results found for: 'Life on Your Knees (2018 Remaster) Neurosis'
Searching for "Plague Bearer" by Claret Ash...
Done.
Searching for "Temptation ov Lost" by Christ Agony...
Done.
Searching for "Essence of Fire" by Claret Ash...
Done.
Searching for "Origin" by Todtgelichter...
No results found for: 'Origin Todtgelichter'
Searching for "Catarctic Cartoons" by Arctic Plateau...
Specified song does not contain lyrics. Rejecting.
Searching for "We Love Because He First Loved Us" by GaidjinN...
Done.
Searching for "Fuigheall" by Cnoc An Tursa...
No results found for: 'Fuigheall Cnoc An Tursa'
Searching for "N.A.F.H." by Selvans...
Done.
Searching for "Summon the Horde" by Asmodeus...
Done.
Searching for "A Handful of Nothing" by Ebony Tears...
Done.
Searching for "The Sixth Bloodletting" by In Reverence...
Done.
Searching for "My Empire" by Deserted Fear...
Done.
Searching for "The Fall of Irminsul" by Rebellion...
Done.
Searching for "My Ulysses" by I Spit Ashes...
Done.
Searching for "Cr

Done.
Searching for "Carthago Delenda Est" by Ancient Rites...
Done.
Searching for "For the Dying Hope" by Vision Eyes...
Done.
Searching for "The Spectre" by Messenger...
Done.
Searching for "Mouth of Madness (Live)" by Tsjuder...
No results found for: 'Mouth of Madness (Live) Tsjuder'
Searching for "Planet Zombie" by Zombie Inc....
Done.
Searching for "Shahanshah" by Viper Solfa...
Done.
Searching for "Bionic Abomination" by Deadborn...
Done.
Searching for "The Newborn" by Furtherial...
Done.
Searching for "Scandinavian Warmachine" by Paganizer...
Done.
Searching for "Speed of God" by Kekal...
Done.
Searching for "Exalting the Shadow Proprietor" by Cara Neir...
Done.
Searching for "Precept of Delator" by Lectern...
No results found for: 'Precept of Delator Lectern'
Searching for "Ruthless Cleansing" by Brutally Deceased...
Done.
Searching for "The Fallen Hero" by Heaven's Scum...
Done.
Searching for "27 Days of Darkness" by Phobocosm...
Done.
Searching for "(We Are The) Doomed" by Ci

Done.
Searching for "Black Widow" by Perséphone...
Done.
Searching for "Ovulation Bitch" by Vendetta...
Done.
Searching for "Last of a Fading Kind" by Deserted Fear...
Done.
Searching for "Still Standing Up" by Bloodhunter...
No results found for: 'Still Standing Up Bloodhunter'
Searching for "Cry. Pray. Die." by Pokerface...
Done.
Searching for "Elisabeth Dane" by The Vision Bleak...
Done.
Searching for "Ashes Remain" by Blaze of Perdition...
Done.
Searching for "Avydia" by Otus...
No results found for: 'Avydia Otus'
Searching for "Mushroom Song" by WAIT...
Done.
Searching for "Nutrient Painting" by Yellow Eyes...
Done.
Searching for "The Awakening Of The Beast" by Death SS...
Done.
Searching for "Driven by Nescience" by Nervecell...
Done.
Searching for "Ageless Existence" by The Ravenous...
Done.
Searching for "Feasting from the Womb" by The Ravenous...
Done.
Searching for "Orgy in Dog's Blood" by The Ravenous...
Done.
Searching for "Perverted Before God" by The Ravenous...
Done.
Sea

Done.
Searching for "Funereal Might" by Arsis...
Done.
Searching for "Clockwise Metamorphosis" by Tomb Mold...
Done.
Searching for "Wicked Fields of Calm" by Leviathan...
Done.
Searching for "Deviant" by Despite Exile...
Done.
Searching for "Sorgen (Sunwheels)" by 1476...
Done.
Searching for "The Valkyries Call" by Stormwarrior...
Done.
Searching for "At the Sound of the Millenium Black Bells - Live" by Enthroned...
Done.
Searching for "Silence" by Catamenia...
Done.
Searching for "Ave Master Lucifer" by Besatt...
Done.
Searching for "Blinding Sun" by Sad Theory...
Done.
Searching for "The Endless Regression of Mind" by Odious Mortem...
Done.
Searching for "Heroes in Godly Blaze" by Adramelech...
Done.
Searching for "Within the Realm of a Dying Sun" by Love Like Blood...
Done.
Searching for "Solace" by Ashbringer...
Done.
Searching for "Golem" by Darkwell...
Done.
Searching for "Night Vision" by Medium...
Done.
Searching for "Flames Of An Endless Disease" by Soulburner...
No results fo

In [189]:
file = '../data/raw/rock_hard_rock_song_lyrics.tsv'
scrape_song_lyrics(rock_hard_rock, genius_access_token, file)

IndexError: single positional indexer is out-of-bounds

In [ ]:
file = '../data/raw/blues_song_lyrics.tsv'
scrape_song_lyrics(blues, genius_access_token, file)

In [179]:
genius.search_song('Gorroneality', 'Avulsed') is None

Searching for "Gorroneality" by Avulsed...
No results found for: 'Gorroneality Avulsed'


True

In [169]:
for row in rock_death_black.index:
    print(rock_death_black.iloc[row].title_name, rock_death_black.iloc[row].artist_name, sep = ' | ')

Onward to Die | Paganizer
Illumina | Exlibris
History Bodies | Amagortis
An Eternal Dark Horizon | Throne Of Katarsis
An Icon for the Damned | Paganizer
Slaughtered Corpse | Amagortis
Lost Children in the World | Opus Doria
Gorroneality | Avulsed
Heraldic | Autumnblaze
Iced | Bodyfarm
Remembrances | Blessed in Sin
Soultheft | Centurian
Bonesaw Bonanza | Paganizer
Departure | Eye of Solitude
Departure of the Stellar Fleet Marks the Year One | Deconstructing Sequence
Zombiefied Electrified | Lay Down Rotten
The Siege of the Mind | Bodyfarm
Let My Blood Boil Tonight | KadavriK
Forever Scorned | Paganizer
Temptation for Hate | Pressure Points
Autodogmatic | Inner Thought
The Frozen Halls | Bodyfarm
Symbol of Night & Winter (Ancient Lords) | Nazxul
Trinity, A New Start | Retribution
Bloodstaind Hands | Disparaged
Stain of Harrow | Nazxul
Murder of Dreams | Gabriel Cyphre
Demons of the Cross | Bodyfarm
World Oblivion | Nazxul
Simple Broken Man | Vortex of Clutter
Ain't No Pit Deep Enough | G

Burning the Deception's Kollapse | Arkha Sva
The Seraphim Veil | Anomalous
A Vision They Shared | Mandragora Scream
A Scenery of Despair | Misfortune
The First Objective | Decaying
Bad Awakening | Kaotik
Bird of Prey | Darkestrah
Postmortem/Raining Blood | Havok
Lost | Darktribe
Withstanding the Funeral | Brutally Deceased
Torn from the Cross | Beastcraft
The Dark Commander | Antalgia
Fiery Torches | Christ Agony
It’s Time | Strained Nerve
Mo(u)rningdance | Autumnblaze
Unearthed | Beneath
Inanimate Diary | Ephemeral Ocean
Strength | Heavenwood
Grey Rigorism | Merrimack
Necromancer | Nexilva
Lightning Guide | Darktribe
Psalm | Devilyn
Contempt | Devilyn
Dubh, Dun Agus Liath | Celtachor
Belial | And There Will Be Blood
Yet I Breathe | Eye of Solitude
Gathering the Stones | Fen
March of Demise | Kormak
Epitaph | Messiah
Not A Nightmare | Artillery
Lurid Assimilation | Defeated Sanity
Divide et Impera | Lahmia
Unplug My Brain | Asylum Pyre
My Last Call | Slechtvalk
It Comes in Whispers | L

The Last Song | Perséphone
Terror of a Perilous Quest | Enfold Darkness
Hate/Aggression Schedule) | Disbelief
The Pleistocene Epoch | Evoken
In Less Than No Time | Mord'a'Stigmata
For the Emperor | Debauchery
10:56 (Audiotree Live Version) | Oathbreaker
And Their Flesh Still Burns | Bombs of Hades
Desire of Destruction | Hevilan
Emet | Stormlord
The Oldest of Dilemmas | Aurora Borealis
The Devil Is Back | Mystic Prophecy
Cursed Possessions | Midnight
Transcendental Paradox | Decrepit Birth
Sliced Open | Carnal Decay
With Madman's Faith | Blaze of Perdition
Shift | Egokills
The Fiends That Come to Steal the Weed of the Deceased | Cannabis Corpse
Fortified Bravery | Sinister
Watchers from Above | Aurora Borealis
Mind Seduction | Disharmonic Orchestra
Times | Mostly Harmless
Scars of a Land Forgotten | Sarcasm
Vortex of the Vultures | Sarcasm
Come Damnation | Keep of Kalessin
Kingdom of Sin | Diabolus Arcanium
The Great Calm Embraced | Sarcasm
Merciless Watcher | Tomb Mold
The Moulting (I

Fire | The Burning
Ink | Ravenscry
Infinity Horizon | Vesania
Haelegen | Liturgy
Frozen Planets | Artificial Brain
Wasted Screams | Bleeding Red
Gutted (Live Bootleg, Berkeley Square 02/03/1997) | Cannibal Corpse
Hagakure | Carved
Swamp | Carved
Camlann | Carved
Heart of Gaia | Carved
God of Disturbance and Friction | Darkthrone
Dead Tomorrow | Construct
Where Black Ships Sail | Sulphur Aeon
Legion | Johnny Deathshadow
Fasciitis Necroticans | MAGEFA
Subzero | Monotheist
Empathy's Greed | Novembers Doom
Low Empire | Plebeian Grandstand
Facing the Faceless | Demonic Resurrection
Oozing Vaginal Discharge | Lividity
From Ruin | Khemmis
Neither of Gods | Throneaeon
Deviant Shapes | White Ward
The Raven and the Crow | Gigan
Artistic Butchery | Brutality
Avoiding the Pain | Icon In Me
Archaic | Malevolent Creation
Procrastination | Precipitation
Unavoidable | Precipitation
Death Upon Lindisfarne | Immorgon
The Sin Offering | Unearthly
The Dove And The Crow | Unearthly
Walldancer | Ivanhoe
All

Telepathic with the Deceased | Xasthur
Hounds | Dark Sermon
Back for Good | Crystal Ball
Your Eyes | Officium Triste
The Thousand Faces of Lies | Death Decline
A Spiral Rift Towards Damnation | Within Destruction
I See, the Sea! | Macabre Omen
My Kantele | Ethereal Kingdoms
To Thou Who Dwellest in the Night | Arcturus
Sodom | Imperial Triumphant
Exile | Domina Noctis
Right to Luck | Nomans Land
Follow in the Cry (Remaster) | After Forever
Lance of Longinus | The Unguided
Tunnel of Trees | Deafheaven
Salomé | Xandria
Razor in Your Back | Exciter
Brighter Than the Sun | Seventh Avenue
Born Treacherous (Live In Oslo) | Dimmu Borgir
Born Dead | Annotations of an Autopsy
Backbreaker | Dyscarnate
Being | Humanity's Last Breath
Forever Mournful | Netherbird
A Grave Robber's Work | The Black Dahlia Murder
Closed Eye Visuals | Meshuggah
Defeated by Life | Happy Days
Midnight | The Birthday Massacre
Born Dead - Live at Backstreets, Rochester, NY - December 13th, 1988 | Death
Black Halo | Secrets

...

Moon Funeral | Artificial Brain
The Lie Vision | Dallian
Jackalian Cry | Legenda
Between the Dying and the Dead | Near Death Condition
Over | Sculpture
Morning Glory Clouds (In Manus Tuas Domine) | Heavenwood
Trail of Blood | Gehenna
Conjuration of Terror | Xasthur
Embracing Waves | Eternal Storm
Ephemeral (Session) [Remaster] | After Forever
Inno a Satana (Alternative Mix 1993) (Previously Unreleased) | Emperor
Soothsayer | Brodequin
Internecine Iatrogenesis | Deathspell Omega
Ivory March | Time For Vultures
Naughty Naughty Naughty | 69 Chambers
Coincidence of Opposites | Tomb Mold
Shades over the Empire | The Shadeless Emperor
Force Majeure | Greytomb
Abhorrence And Anxiety | Necrotted
Liberty, Equality, Brutality | Necrotted
On Death Row | Eraserhead
Firestorm | Ran
Mortal Decimation | Skeletal Remains
Tears, Hope and Hate | Krabathor
Infernal Desire (Live) | Malevolent Creation
Svartheme | Immorgon
Just Another | Skymning
I Am God | Omega Lithium
Locked Out | Scanner
Moth Defect (B

Refuse / Resist (Remastered) | Sepultura
Counrting Moments | Darkseed
De Masticatione Mortuorum | Azaghal
Splinters from Your Mother's Spine | Deathspell Omega
Heavenly Perverse | Dimmu Borgir
Where the Enemy Sleeps... (Remastered) | Kataklysm
Manifesto | Imperial Triumphant
Killing Tendencies | Havok
Heathenpride | Falkenbach
Act IV: The Pathway Had Been Lost | Eye of Solitude
Bloodwork | Graveworm
Obsidian Night | Jex Thoth
Hold Your Head Up | Clawfinger
The Taste of Iron | Martyr Defiled
The Bells | Aeternitas
Devotion to the Cosmic Chaos | Sulphur Aeon
Laments of Fear & Despair (Remastered) | Kataklysm
Cross the Threshold | Loudblast
It's Getting Unclear | Loser First
The Secret | Ashes of Moon
I Am the Beast | Okkultist
C.I.U. (Criminals In Uniform) | Sepultura
Obscura Symphonia | Massemord
Ruun | Enslaved
The Love I Hate | Disharmonic Orchestra
Rise of the Light Bearer | Sinsaenum
Sign from the North Side (The Oulu Show Live) | Amorphis
Retarded Phrase Mongers | Monument Of Misan

The Bard's Song: In the Forest (Live, Remastered 2007) | Blind Guardian
I Bear Fire | Ephemerald
Into the Dark | Mors Principium Est
Towards the Twilight | Night In Gales
Hearselights | Night In Gales
Inanimate | Abysmal Dawn
Rest in Peace | Rebellion
Being Everyone | After Forever
The Little Big Man | Asp
Ad Absurdum (Live) | Arcturus
Frontiers | Omnium Gatherum
Wake Up - Live | Suicide Silence
Withering Strands | Be´lakor
Ira | Aavas
In the Halls of Awaiting | Insomnium
Closing Words | Insomnium
Of Dementia | The Great Old Ones
Undone | Craft
Infintesimal | Evoke Efrits
Mind's Mirrors | Meshuggah
Troops Of Doom - Schizophrenia Bonus Track [Reissue] | Sepultura
Episode | Rivers of Nihil
Follower | Abomination
Antigone | Aephanemer
Affluenza | Hadal Maw
Hold Me up, Hold Me Down | Suicide Silence
Hedonist | Re-Armed
Hate for Your God | The Duskfall
Ivory Towers | Re-Armed
Iron Mule | Avatarium
In Sanctum | Siebenbürgen
Unleash Hell | Malevolent Creation
Alive? | November-7
Focus | River

Got Ist Tott | Demondeath
Starless Event Horizon | Corpsessed
Abysmal Structures | Syndemic
Weeping Ghost | Angellore
Tastes Like Murder | Port Of Call
Lacrima Caelestis | Nightshade
Prelude (Live) | Laaz Rockit
Absolve Me | I Shalt Become
Sky Shooting Stars | Enemy of the Sun
Quiet | Ruined Families
Violation Of The Soul | Sear
Night Song (Apocalyptic Dawn) | Demoncy
Almost Dead | Pandemmy
Inferno Is Over | Pandemmy
The Filthmonger | Chthonian
An Oath Sworn in Sorrow | Sojourner
Declined Sameness | Sad Theory
A Torch Spreads Its Coldness | Sad Theory
Arming For Retribution | Kambrium
Stevenology | Fernando Colunga Ultimate Experience
Religious Fantasy | Godgory
The Eternal Validity | Disavowed
Calculated Barbarity | Defeated Sanity
A Thought, Ablaze | Dawn Ray'd
Pulse of an End | Ad Cinerem
The Sacred Daemonic | Enfold Darkness
A Lifetime | Count Raven
Obsidian Chromatism | Xenoblight
End Of Eternity | Artillery
Wargrinder | Debauchery
Bloody Pit of Horror | Lividity
Amaurosis | Synde

The Valkyries Call | Stormwarrior
At the Sound of the Millenium Black Bells - Live | Enthroned
Silence | Catamenia
Ave Master Lucifer | Besatt
Blinding Sun | Sad Theory
The Endless Regression of Mind | Odious Mortem
Heroes in Godly Blaze | Adramelech
Within the Realm of a Dying Sun | Love Like Blood
Solace | Ashbringer
Golem | Darkwell
Night Vision | Medium
Flames Of An Endless Disease | Soulburner
Last Dance of a Treacherous Mind | Nightland
Drowning in Sorrow | Inner Thought
Cradle of Sufferance | Nightland
Primitive Again | Falchion
In Hell I Burn | Slechtvalk
A Prophet for a Pound of Flesh | A Forest of Stars
Chapter III. The Annunciation in Lust | Dark Mirror Ov Tragedy
Partly Alive | The Body
No Dust | Yellow Eyes
Dead Eyes | Wheelfall
Pilgrimage | Wheelfall
El Drago-Un | Lucifer's Child
Kingdom of Abyss | Christ Agony
Dark Goddes | Christ Agony
Smithereens / Smoke (The Oulu Show Live) | Amorphis
Vicious Minds | Dark Void
Eye Sockets Empty | Centinex
Radiation Damage | Human Debr

In [109]:
genius_access_token = Keys.genius_access_token

In [14]:
from lyricsgenius import Genius

In [110]:
genius = Genius(genius_access_token)

In [145]:
song = genius.search_song('Red Gold & Green', 'Dian Marley')

Searching for "Red Gold & Green" by Dian Marley...


Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [18]:
song.lyrics

"Red, Gold & Green Lyrics(Chorus: Damian Marley and Kabaka Pyramid)\nRed Gold and Green\nRed Gold and Green\nWave the banner from yuh know yuh soul clean\nRed Gold and Green\nRed Gold and Green\nRoots reggae music up inna yuh blood stream\nRed Gold and Green\nRed Gold and Green\nEvery corner, every lane, every scheme\nRed Gold and Green\nRed Gold and Green\nYes unnu fi know\n(Verse 1 : Kabaka Pyramid)\nThis a di energy the people dem been lacking\nThe reggae beat dis a nuh trapping\nAll who feel like dem a tapping\nReally sleeping and dem napping\nWhile mi flying roun di world\nThink man a genie like Aladdin\nAnd for water down the message just for streaming I'm not having\nPeople inna foreign seh dem nah get nuh good music\nSo mi gwaan inna di booth and then record\nLike man a cupid\nHit the target when mi shoot it\nIt nuh hard fi execute dis\nLyrics God mi done tell yuh fi watch mi prove it\n\n(Chorus: Damian Marley and Kabaka Pyramid)\nRed Gold and Green\nRed Gold and Green\nWave th

In [47]:
song = genius.search_song('Infernal Death', 'Death')

Searching for "Infernal Death" by Death...
Done.


In [53]:
song.lyrics

'Infernal Death Lyrics[Intro]\nDie, die, die\n\n[Verse 1]\nPiling the bodies\nBurn them in the night\nSkin grows black and withered\nDecayed smell will rise\n\n[Chorus]\nExistence fading\nInto ashes\nBurn those bodies\nTo Infernal death\n[Verse 2]\nHuman coals are burning\nRepulsive yet so true\nOpen graves are scattered\nWhen the work is through\n\n[Chorus]\nExistence fading\nInto ashes\nBurn those bodies\nTo Infernal death\n\n[Chorus]\nExistence fading\nInto ashes\nBurn those bodies\nTo Infernal deathYou might also likeEmbed'

In [48]:
song_split = song.lyrics.split('\n')

In [54]:
song_split_no_empty_headers = [x for x in song_split if '(' not in x and x != '' and '[' not in x]

In [55]:
song_split_no_empty_headers

['Die, die, die',
 'Piling the bodies',
 'Burn them in the night',
 'Skin grows black and withered',
 'Decayed smell will rise',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal death',
 'Human coals are burning',
 'Repulsive yet so true',
 'Open graves are scattered',
 'When the work is through',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal death',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal deathYou might also likeEmbed']

In [68]:
song_scraped_artefacts_removed = []

for line in song_split_no_empty_headers:
    song_scraped_artefacts_removed.append(re.sub(r'([A-Z]?[a-z]+)[A-Z][\w\W]+', '\g<1>', line))

In [146]:
song_scraped_artefacts_removed

['Die, die, die',
 'Piling the bodies',
 'Burn them in the night',
 'Skin grows black and withered',
 'Decayed smell will rise',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal death',
 'Human coals are burning',
 'Repulsive yet so true',
 'Open graves are scattered',
 'When the work is through',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal death',
 'Existence fading',
 'Into ashes',
 'Burn those bodies',
 'To Infernal death']

In [69]:
for index in range(len(song_split_no_empty_headers)):
    if song_split_no_empty_headers[index] != song_scraped_artefacts_removed[index]:
        print('Index: {}\nOld: {}\nNew: {}'.format(index,
                                                   song_split_no_empty_headers[index],
                                                   song_scraped_artefacts_removed[index]))

Index: 20
Old: To Infernal deathYou might also likeEmbed
New: To Infernal death


In [112]:
genius.search_song('Don\'t Let me Go', 'Harrys Cupboard').lyrics

Searching for "Don't Let me Go" by Harrys Cupboard...
Done.


'Home Alone (Script) Lyrics(SCENE: HARRY IS DRESSED IN A POLICE OFFICER\'S UNIFORM; McCALLISTER FAMILY IS QUICKLY ENTERING AND EXITING)\n\nHARRY:\nMiss. Young lady! Excuse me. Girls! Hey, hey, little guy. Hey! Excuse me, girls. Girls! Hey, big fella! Excuse me. Ma\'am, excuse me. Ma\'am! Ma\'am! Hey, son. Son! Big fella. Hey, hey hey, little guy, little guy!\n\nHelp me make the beds in the living room.\n\nCome on down here!\n\nHey, son!\n\nBig fella. Hey, little guy! Little guy!\n\nPete\'s brother and his family are here.\n\nTrish is going to Montreal.\n\nMontreal? Oh, her family\'s there.\n\n- Then we\'re off. - When?\n\n- Tomorrow. - You\'re not ready, are you?\n\nKEVIN:\nMom, Uncle Frank won\'t let me watch the movie, but the big kids can. Why can\'t I?\nKATE:\nKevin, I\'m on the phone. (Back on the phone) When do you come back? Not \'til then?\n\nKEVIN:\nIt\'s not even rated R. He\'s just being a jerk.\n\nKATE:\nKevin, if Uncle Frank says no, then it must be really bad. No, we put 